In [21]:
import sys
from pathlib import Path

# assume this notebook lives in notebooks/, so parent() is the repo root
sys.path.append(str(Path().resolve().parent))
from paths import PROJECT_ROOT

In [22]:
import numpy as np
import pandas as pd
from pathlib import Path

# DreaMS imports
from dreams.utils.dformats import DataFormatA
from dreams.utils.data     import MSData
from dreams.utils.io       import append_to_stem

In [23]:
# 2) point to your file (mzML or HDF5)
mzml_path = PROJECT_ROOT / "data" / "rawfiles" / "202312_20_P09-Leaf-r1_1uL.mzML"
h5_path   = mzml_path.with_suffix(".hdf5")

In [24]:
# 3) load (or reuse cache) into an MSData
if h5_path.exists():
    msdata = MSData.from_hdf5(str(h5_path), in_mem=True)
else:
    msdata = MSData.from_mzml(str(mzml_path))


Loading dataset 202312_20_P09-Leaf-r1_1uL into memory (5681 spectra)...


In [25]:

# 2) filter to positive‐ion spectra only ---------
charges = msdata.get_values("charge")       # (N,)
pos_idx = np.where(charges > 0)[0]

print(f"Keeping {len(pos_idx)} / {len(charges)} spectra with charge > 0")

# 3) subset your MSData in‐memory (no extra disk write)
#    (we’ll just reindex all further get_spectra(), get_prec_mzs(), etc.)
msdata_pos = msdata.form_subset(pos_idx, out_pth=append_to_stem(h5_path, "positive"))

Keeping 4562 / 5681 spectra with charge > 0


In [29]:
# 4) pull out spectra & precursor m/z arrays -----
spectra  = msdata_pos.get_spectra()    # shape: (M, 2, P)
prec_mzs  = msdata_pos.get_prec_mzs()  # shape: (M,)


In [30]:
spectra.shape

(4562, 2, 128)

In [31]:
# 5) compute single‐spectrum quality flags
dformat      = DataFormatA()
quality_lvls = [
    dformat.val_spec(spec, prec, return_problems=True)
    for spec, prec in zip(spectra, prec_mzs)
]

/Users/macbook/UTILS/anaconda3/envs/dreams_mimb/lib/python3.11/site-packages/dreams/utils/spectra.py:258: RuntimeWarning: divide by zero encountered in scalar divide
  return max(peak_list[1]) / min(peak_list[1])


In [32]:
# 6) tally pass/fail
qc_counts = pd.Series(quality_lvls).value_counts()
print("QC breakdown:\n", qc_counts)

QC breakdown:
 All checks passed                      3741
Number of high intensity peaks >= 3     785
Intensity amplitude >= 20.0              21
m/z range <= 1000.0                      15
Name: count, dtype: int64


In [33]:
# 7) select only 'All checks passed'
keep_idx = np.where(np.array(quality_lvls) == "All checks passed")[0]

In [35]:
# 8) write high‐quality subset to new HDF5
hq_h5 = append_to_stem(h5_path, "high_quality")
msdata.form_subset(idx=keep_idx, out_pth=hq_h5)
print("Wrote high-quality file to:", hq_h5)

Wrote high-quality file to: /Users/macbook/CODE/DreaMS_MIMB/data/rawfiles/202312_20_P09-Leaf-r1_1uL_high_quality.hdf5


In [36]:
# 9) verify by loading it back
msdata_hq = MSData.load(hq_h5)
print("High-quality MSData:", msdata_hq)

High-quality MSData: MSData(pth=/Users/macbook/CODE/DreaMS_MIMB/data/rawfiles/202312_20_P09-Leaf-r1_1uL_high_quality.hdf5, in_mem=False) with 3,741 spectra.


In [37]:
msdata.get_values("charge")

array([0, 1, 1, ..., 0, 1, 1])